In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import gc
import glob

# 그래프 기본 테마 설정
# https://coldbrown.co.kr/2023/07/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%EC%8B%A4%EC%A0%84%ED%8E%B8-08-seaborn-sns-set%EC%9D%84-%ED%86%B5%ED%95%B4-%EC%8A%A4%ED%83%80%EC%9D%BC-%EC%84%A4%EC%A0%95%ED%95%98%EA%B8%B0/
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False


# 복잡한 통계 처리를 위한 라이브러리
from scipy import stats

In [2]:
df1 = pd.read_csv('data/KBO FA 투수 [2013-2024]_최종_전처리+파생.csv', encoding='utf_8')

In [3]:
df2 = pd.read_csv('data/파크팩터.csv', encoding='cp949')

In [4]:
df3 = pd.read_csv('data/국대출전선수.csv', encoding='cp949')

In [5]:
df4 = pd.read_csv('data/구종가치.csv', encoding='utf-8')

In [6]:
df1.columns

Index(['FA년도', '구단명', '선수명', '세부 포지션', '투', '구분', 'FA등급', '플레이년도', '나이', '게임수',
       '선발등판', '구원등판', '승리', '패전', '세이브', '블론세이브', '홀드', '블론홀드', '이닝수', '자책점',
       '피안타', '2루타허용', '3루타허용', '피홈런', '볼넷허용', '사구허용', '탈삼진', '폭투', 'ERA',
       'FIP', '선발 이닝', '구원 이닝', '선발 WAR', '구원 WAR', '종합 WAR', '연봉', '비고1',
       '잔류 여부', 'FA 계약 연수', 'FA 계약 총액', '사사구', '완투', '완봉', '투구수', '피장타율',
       '원클럽맨여부'],
      dtype='object')

In [7]:
df1['선수명'].unique()

array(['강영식', '강윤구', '고효준', '권오준', '권혁', '금민철', '김광현', '김대우', '김사율',
       '김상수', '김승회', '김재윤', '김진성', '노경은', '마일영', '마정길', '박정진', '배영수',
       '백정현', '봉중근', '손승락', '송승준', '송은범', '심수창', '안영명', '안지만', '양현종',
       '오승환', '오주원', '우규민', '원종현', '유동훈', '유희관', '윤규진', '윤길현', '윤석민',
       '윤성환', '이동현', '이보근', '이용찬', '이재영', '이재학', '이정훈', '이태양', '이현승',
       '임찬규', '임창민', '장민재', '장시환', '장원삼', '장원준', '정우람', '정찬헌', '정현욱',
       '주권', '진해수', '차우찬', '채병용', '한현희', '함덕주', '홍건희', '문승원', '박종훈',
       '박세웅', '류현진', '고영표', '구창모'], dtype=object)

In [9]:
# 파크팩터 병합
merge_cols = ['플레이년도', '구단명']
factor_cols = ['득점파크팩터', '홈런파크팩터', '장타파크팩터']

# 병합 수행 (left join: df1 기준, df2 정보 가져오기)
merged_df = pd.merge(df1, df2[merge_cols + factor_cols], on=merge_cols, how='left')

# 결과 확인
print(merged_df.head())

   FA년도 구단명  선수명 세부 포지션   투      구분 FA등급  플레이년도    나이   게임수  ...  FA 계약 총액  \
0  2011  롯데  강영식     RP  좌투    자격유지    A   2010  29.0  63.0  ...       0.0   
1  2012  롯데  강영식     RP  좌투    자격유지    A   2011  30.0  64.0  ...       0.0   
2  2013  롯데  강영식     RP  좌투    자격유지    A   2012  31.0  55.0  ...       0.0   
3  2014  롯데  강영식     RP  좌투    자격유지    A   2013  32.0  55.0  ...  170000.0   
4  2015  롯데  강영식     RP  좌투  FA 1년차    A   2014  33.0  52.0  ...       0.0   

    사사구  완투  완봉  투구수   피장타율  원클럽맨여부  득점파크팩터  홈런파크팩터  장타파크팩터  
0  25.0   0   0  907  0.412     0.0  1032.0  1128.0  1093.0  
1  24.0   0   0  731  0.360     0.0  1044.0  1128.0  1093.0  
2  19.0   0   0  757  0.340     0.0  1033.0  1128.0  1093.0  
3  21.0   0   0  675  0.298     0.0  1058.0  1128.0  1093.0  
4  26.0   0   0  813  0.393     0.0  1045.0  1102.0  1062.0  

[5 rows x 49 columns]


In [11]:
#미국 선수들 파크팩터는 nan
merged_df['득점파크팩터'].isna().sum()

6

In [12]:
merged_df['홈런파크팩터'].isna().sum()

6

In [13]:
merged_df['장타파크팩터'].isna().sum()

6

In [14]:
#df_merged.to_csv('파크팩터 추가.csv')

In [15]:
#국대출전횟수 병합 및 처리
merged_df = merged_df.merge(df3, on='선수명', how='left')
merged_df['국대출전횟수'] = merged_df['국대출전횟수'].fillna(0).astype(int)

print(merged_df)

     FA년도 구단명  선수명 세부 포지션   투      구분  FA등급  플레이년도    나이   게임수  ...   사사구  완투  \
0    2011  롯데  강영식     RP  좌투    자격유지     A   2010  29.0  63.0  ...  25.0   0   
1    2012  롯데  강영식     RP  좌투    자격유지     A   2011  30.0  64.0  ...  24.0   0   
2    2013  롯데  강영식     RP  좌투    자격유지     A   2012  31.0  55.0  ...  19.0   0   
3    2014  롯데  강영식     RP  좌투    자격유지     A   2013  32.0  55.0  ...  21.0   0   
4    2015  롯데  강영식     RP  좌투  FA 1년차     A   2014  33.0  52.0  ...  26.0   0   
..    ...  ..  ...    ...  ..     ...   ...    ...   ...   ...  ...   ...  ..   
360  2019  NC  구창모     SP  좌투    다년계약  다년계약   2018  21.0  36.0  ...  59.0   0   
361  2020  NC  구창모     SP  좌투    다년계약  다년계약   2019  22.0  23.0  ...  47.0   0   
362  2021  NC  구창모     SP  좌투    다년계약  다년계약   2020  23.0  15.0  ...  18.0   0   
363  2023  NC  구창모     SP  좌투    다년계약  다년계약   2022  25.0  19.0  ...  33.0   0   
364  2024  NC  구창모     SP  좌투  FA 1년차  다년계약   2023  26.0  11.0  ...  16.0   0   

     완봉   투구수   피장타율  원클럽맨여

In [16]:
merged_df = merged_df.merge(df4[['선수명', '플레이년도', '구종1', '구종2', '구종1 구종가치']],
                      on=['선수명', '플레이년도'], how='left')

In [17]:
merged_df

,FA년도,구단명,선수명,세부 포지션,투,구분,FA등급,플레이년도,나이,게임수,...,투구수,피장타율,원클럽맨여부,득점파크팩터,홈런파크팩터,장타파크팩터,국대출전횟수,구종1,구종2,구종1 구종가치
0,2011,롯데,강영식,RP,좌투,자격유지,A,2010,29.0,63.0,...,907,0.412,0.0,1032.0,1128.0,1093.0,0,NaN,NaN,NaN
1,2012,롯데,강영식,RP,좌투,자격유지,A,2011,30.0,64.0,...,731,0.360,0.0,1044.0,1128.0,1093.0,0,NaN,NaN,NaN
2,2013,롯데,강영식,RP,좌투,자격유지,A,2012,31.0,55.0,...,757,0.340,0.0,1033.0,1128.0,1093.0,0,NaN,NaN,NaN
3,2014,롯데,강영식,RP,좌투,자격유지,A,2013,32.0,55.0,...,675,0.298,0.0,1058.0,1128.0,1093.0,0,포심,슬라,5.0
4,2015,롯데,강영식,RP,좌투,FA 1년차,A,2014,33.0,52.0,...,813,0.393,0.0,1045.0,1102.0,1062.0,0,포심,슬라,-5.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,2019,NC,구창모,SP,좌투,다년계약,다년계약,2018,21.0,36.0,...,2383,0.483,1.0,1004.0,1093.0,992.0,1,포심,커브,2.3
361,2020,NC,구창모,SP,좌투,다년계약,다년계약,2019,22.0,23.0,...,1780,0.338,1.0,1043.0,1084.0,1018.0,1,포심,슬라,17.8
362,2021,NC,구창모,SP,좌투,다년계약,다년계약,2020,23.0,15.0,...,1378,0.278,1.0,1037.0,1091.0,1026.0,1,포심,슬라,21.9
363,2023,NC,구창모,SP,좌투,다년계약,다년계약,2022,25.0,19.0,...,1756,0.314,1.0,1013.0,1095.0,1026.0,1,포심,슬라,23.4


In [18]:
merged_df.columns

Index(['FA년도', '구단명', '선수명', '세부 포지션', '투', '구분', 'FA등급', '플레이년도', '나이', '게임수',
       '선발등판', '구원등판', '승리', '패전', '세이브', '블론세이브', '홀드', '블론홀드', '이닝수', '자책점',
       '피안타', '2루타허용', '3루타허용', '피홈런', '볼넷허용', '사구허용', '탈삼진', '폭투', 'ERA',
       'FIP', '선발 이닝', '구원 이닝', '선발 WAR', '구원 WAR', '종합 WAR', '연봉', '비고1',
       '잔류 여부', 'FA 계약 연수', 'FA 계약 총액', '사사구', '완투', '완봉', '투구수', '피장타율',
       '원클럽맨여부', '득점파크팩터', '홈런파크팩터', '장타파크팩터', '국대출전횟수', '구종1', '구종2',
       '구종1 구종가치'],
      dtype='object')

In [19]:
merged_df.to_csv('KBO FA 투수 [2013-2024]_최종_전처리+파생완료.csv')